In [1]:
from haystack.document_stores.in_memory import InMemoryDocumentStore
document_store = InMemoryDocumentStore()

DATASET FECTCING DIRECTLY FROM HUGGING FACE 

DATASET KEY = bilgeyucel/seven-wonders

In [23]:
from datasets import load_dataset
from haystack import Document

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]



In [24]:
from haystack.components.embedders import SentenceTransformersDocumentEmbedder

doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
doc_embedder.warm_up()

In [25]:
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches: 100%|██████████| 5/5 [00:08<00:00,  1.70s/it]


DuplicateDocumentError: ID '75fd8474f2c88337f7e0dad69eba0f24ba293cb06693fb746ec403df01a1c0c5' already exists.

FOR USER QUERY I HAVE TO CREATE A TEXT EMBEDDER(ALSO DEFINED IN THE HAYSTACK DOCUMENTATION)

In [6]:
from haystack.components.embedders import SentenceTransformersTextEmbedder

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")


In [7]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

retriever = InMemoryEmbeddingRetriever(document_store)

In [8]:
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage

template = [
    ChatMessage.from_user(
    """Given the following information, answer the question.
    
    comtext:
    {% for document in documents %}
        {{ document.content}}
    {% endfor %}
    
    Question: {{question}}
    Answer:
    """
    )
]

prompt_builder = ChatPromptBuilder(template=template)

ChatPromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [10]:
import os
from getpass import getpass
from haystack.components.generators.chat import OpenAIChatGenerator

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key:")
chat_generator = OpenAIChatGenerator(model="gpt-4o-mini")


CREATING PIPELING COMPONENTS


In [11]:
from haystack import Pipeline

rag_pipeline = Pipeline()

rag_pipeline.add_component("text_embedder",text_embedder)
rag_pipeline.add_component("retriever",retriever)
rag_pipeline.add_component("prompt_builder",prompt_builder)
rag_pipeline.add_component("llm",chat_generator)


CONNECTING COMPONENT WITH THE RAG PIPELINE

In [12]:
rag_pipeline.connect("text_embedder.embedding","retriever.query_embedding")
rag_pipeline.connect("retriever","prompt_builder")
rag_pipeline.connect("prompt_builder.prompt","llm.messages")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: ChatPromptBuilder
  - llm: OpenAIChatGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.messages (List[ChatMessage])

CREATING EXAMPLE QUESTIONS


In [13]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]


In [26]:
question = "Why did people visit the Temple of Artemis?"

res = rag_pipeline.run({"text_embedder":{"text":question},"prompt_builder":{"question":question}})
# print(res)
# Extract the text
ans = res['llm']['replies'][0]._content[0].text
print(ans)

Batches: 100%|██████████| 1/1 [00:00<00:00, 62.72it/s]


People visited the Temple of Artemis for several reasons:

1. **Religious Significance**: The temple was dedicated to the goddess Artemis, and it served as a major religious site where worshippers paid homage and sought the goddess's protection. It was a place for offerings, prayers, and participation in various religious rituals.

2. **Cultural Events and Festivals**: The temple was the center of the Ephesian Artemis festival (Artemisia), one of the largest and most significant religious festivals in Ephesus. This festival attracted large crowds, bringing together people for games, contests, theatrical performances, and processions in honor of Artemis.

3. **Sanctuary**: The temple offered sanctuary to individuals fleeing persecution or punishment, which enhanced its reputation as a place of safety and refuge.

4. **Tourism and Sightseeing**: The Temple of Artemis was considered one of the Seven Wonders of the Ancient World. Its grandeur and architectural beauty drew merchants, kings,